In [41]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [42]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
updated_app_df = application_df.drop(['EIN', 'NAME'], axis=1)
updated_app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [43]:
# Determine the number of unique values in each column.
updated_app_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [44]:
# Look at APPLICATION_TYPE value counts for binning
updated_app_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [45]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    updated_app_df['APPLICATION_TYPE'] = updated_app_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
updated_app_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [46]:
# Look at CLASSIFICATION value counts for binning
updated_app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [47]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list (updated_app_df['CLASSIFICATION'].value_counts()[updated_app_df['CLASSIFICATION'].value_counts()<700].index)

# Replace in dataframe
for cls in classifications_to_replace:
    updated_app_df['CLASSIFICATION'] = updated_app_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
updated_app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [48]:
# Look at ASK_AMOUNT value counts for binning
updated_app_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [49]:
# Choose a cutoff value and create a list of ASK AMOUNTS to be replaced
# use the variable name `ask_amount_to_replace`
income_amount_to_replace = list (updated_app_df['INCOME_AMT'].value_counts()[updated_app_df['INCOME_AMT'].value_counts()<1000].index)

# Replace in dataframe
for inc in income_amount_to_replace:
    updated_app_df['INCOME_AMT'] = updated_app_df['INCOME_AMT'].replace(inc,"Other")
    
# Check to make sure binning was successful
updated_app_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [50]:
# Convert categorical data to numeric with `pd.get_dummies`
updated_app_df = pd.get_dummies(updated_app_df,dtype=float)
updated_app_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [51]:
# Split our preprocessed data into our features and target arrays
X = updated_app_df.drop('IS_SUCCESSFUL', axis=1).values
y = updated_app_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [52]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Step 3: Optimize the Model
Using your knowledge of TensorFlow, optimize your model to achieve a target predictive accuracy higher than 75%.
Using any or all of the following methods to optimize your model:

Adjust the input data to ensure that no variables or outliers are causing confusion in the model, such as:

Dropping more or fewer columns.
Creating more bins for rare occurrences in columns.
Increasing or decreasing the number of values for each bin.


Add more neurons to a hidden layer.
Add more hidden layers.
Use different activation functions for the hidden layers.
Add or reduce the number of epochs to the training regimen.

In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=39))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 320       
                                                                 
 dense_13 (Dense)            (None, 8)                 72        
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [56]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6180 - accuracy: 0.6602
Epoch 2/100
804/804 [==============================] - 1s 995us/step - loss: 0.5690 - accuracy: 0.7185
Epoch 3/100
804/804 [==============================] - 1s 967us/step - loss: 0.5636 - accuracy: 0.7221
Epoch 4/100
804/804 [==============================] - 1s 903us/step - loss: 0.5606 - accuracy: 0.7238
Epoch 5/100
804/804 [==============================] - 1s 915us/step - loss: 0.5588 - accuracy: 0.7260
Epoch 6/100
804/804 [==============================] - 1s 910us/step - loss: 0.5578 - accuracy: 0.7257
Epoch 7/100
804/804 [==============================] - 1s 944us/step - loss: 0.5563 - accuracy: 0.7261
Epoch 8/100
804/804 [==============================] - 1s 871us/step - loss: 0.5553 - accuracy: 0.7265
Epoch 9/100
804/804 [==============================] - 1s 913us/step - loss: 0.5544 - accuracy: 0.7276
Epoch 10/100
804/804 [==============================] - 1s 948us/step - los

804/804 [==============================] - 1s 884us/step - loss: 0.5469 - accuracy: 0.7315
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7318
Epoch 82/100
804/804 [==============================] - 1s 909us/step - loss: 0.5466 - accuracy: 0.7320
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7305
Epoch 84/100
804/804 [==============================] - 1s 903us/step - loss: 0.5467 - accuracy: 0.7317
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7308
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7322
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7312
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7318
Epoch 89/100
804/804 [==============================] - 1s 952us/step - loss: 0.5463 - ac

In [58]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5560 - accuracy: 0.7282 - 281ms/epoch - 1ms/step
Loss: 0.5560291409492493, Accuracy: 0.7281632423400879


In [59]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")